In [1]:
import solana
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana.transaction import TransactionInstruction, Transaction, AccountMeta
import argparse
from solana.publickey import PublicKey
from solana.blockhash import Blockhash
from solana.system_program import *
import base58, base64
from solana.sysvar import SYSVAR_RENT_PUBKEY
from solana.system_program import SYS_PROGRAM_ID
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.client import Token
import time

In [2]:
client = Client("https://api.devnet.solana.com")
ID = "Fni4pJzEwY3sCE8MAVQFHbXdqm1vEMm85o1o8ormWUJW"
#ID = "A4vEBwVMoEZ8j4gthtMnm3MLgKtZoSrw7kYGq4KotE2Q"#"A4vEBwVMoEZ8j4gthtMnm3MLgKtZoSrw7kYGq4KotE2Q"
program_id=base58.b58decode(ID)

In [3]:
k = [239,114,243,183,37,213,148,161,13,172,25,86,143,240,175,157,215,108,156,181,161,230,0,40,228,255,23,209,52,254,120,234,234,209,190,175,172,89,1,135,75,124,43,221,38,148,22,210,210,175,41,10,176,56,96,41,166,103,76,50,58,78,15,74]
God = Keypair.from_secret_key(bytes(k))

In [4]:
God.public_key

GoduExX7V8SpBDTyf2pRukQgEFV8dXJjpaRuNRQAbfhb

In [5]:
Alice = Keypair()
Bob = Keypair()

In [6]:
client.request_airdrop(Alice.public_key, 1000000000)

{'jsonrpc': '2.0',
 'result': '46xCgHWaAs9ZVx4cZzPX82FWypYkoi96JFETcgbz7m6Q6WGiEMRZQv1kfRfmW32aZXDzDWudKLXAx6KFvqxZ1FF2',
 'id': 1}

In [7]:
Alice.public_key

2qQZBMn5t2xGMmTCuCbr6yvmAzjN6WNUnNiTnkuVAi8Q

In [8]:
X_mint = Token.create_mint(client, God, God.public_key, 9, TOKEN_PROGRAM_ID)
Y_mint = Token.create_mint(client, God, God.public_key, 9, TOKEN_PROGRAM_ID)

In [9]:
token_program_pk = TOKEN_PROGRAM_ID
x_seed = bytes(X_mint.pubkey)
y_seed = bytes(Y_mint.pubkey)
alice_seed = bytes(Alice.public_key)
bob_seed = bytes(Bob.public_key)
X_vault_pk, _ = PublicKey.find_program_address([x_seed, alice_seed, bob_seed], program_id)
Y_vault_pk, _ = PublicKey.find_program_address([y_seed, alice_seed, bob_seed], program_id)
x_vault_seed = bytes(X_vault_pk)
y_vault_seed = bytes(Y_vault_pk)
escrow_pk, _ = PublicKey.find_program_address([x_vault_seed, y_vault_seed, alice_seed, bob_seed], program_id)

In [10]:
account_metas = [
AccountMeta(Alice.public_key, True, False),
AccountMeta(Bob.public_key, False, False),
AccountMeta(X_mint.pubkey, False, False),
AccountMeta(Y_mint.pubkey, False, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(Y_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(SYS_PROGRAM_ID, False, False),
AccountMeta(SYSVAR_RENT_PUBKEY, False, False),
]

In [11]:
idx = 0
size_A = 10
size_B = 1
instruction_data = idx.to_bytes(1,"little") + size_A.to_bytes(8,"little") + size_B.to_bytes(8,"little")

In [12]:
instruction = TransactionInstruction(data=instruction_data,program_id=ID,keys=account_metas)
tix = Transaction().add(instruction)

In [13]:
# initialization
y = client.send_transaction(tix, Alice)
print(y)

{'jsonrpc': '2.0', 'result': 'bohgAkA324HLragiGnSXQPyPDdtJDzooj9DKF9xhLtdLijaTk3gLoKUrAdqiam6pvEYgZwpj7PUAsGxgp7trRNB', 'id': 17}


In [14]:
# DEPOSIT
# create ATAs
X_a = X_mint.create_associated_token_account(Alice.public_key)
Y_a = Y_mint.create_associated_token_account(Alice.public_key)
X_b = X_mint.create_associated_token_account(Bob.public_key)
Y_b = Y_mint.create_associated_token_account(Bob.public_key)

In [15]:
X_a

AB2YhmCidVPUiHaDmJRxj9hss7CJ2iMNz5Du7HzLTCk7

In [23]:
X_mint.mint_to(X_a, God, 500)
Y_mint.mint_to(Y_b, God, 500)

{'jsonrpc': '2.0',
 'result': 'yJvG1xULk5w5bLJ3Qgzia17W39V6r7BxWaSBTzsJiA4SXNCcHgY3xrFd97rQtenMg33MYYr7mySRkz5BFsMDopB',
 'id': 47}

In [17]:
deposit_metas = [
AccountMeta(Alice.public_key, True, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]

In [18]:
idx = 1
amount_a = 10
deposit_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")

In [19]:
instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
tix = Transaction().add(instruction)

In [24]:
y = client.send_transaction(tix, Alice)
print(y)

{'jsonrpc': '2.0', 'result': '2eXtfxPCN39NkpK4RQEyKgiSaEVozcLBWRr7zQLZMvx8kjWu69rksNhfN1AMkiyfPM25xE7uCoGUPBQMoQDjPmVr', 'id': 49}


In [21]:
X_a

AB2YhmCidVPUiHaDmJRxj9hss7CJ2iMNz5Du7HzLTCk7

In [25]:
X_vault_pk

9UqemSG5KjANK3XQnAMQ5EHAgdtoC2tVkZcASXKkLgfF

In [40]:
# WITHDRAW
withdraw_metas = [
AccountMeta(Alice.public_key, False, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]

In [34]:
idx = 2
amount_a = 10
withdraw_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")

In [41]:
instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
tix = Transaction().add(instruction)

In [38]:
escrow_pk

8gmHBXKMGsnSPa3RKwFDJdR2QzDe9ZfQpMuGxAEhBtfM

In [ ]:
y = client.send_transaction(tix, Alice)

## TESTS

In [ ]:
# Alice, Bob both deposit and then withdraw
tix = Transaction()
X_mint.mint_to(X_a, God, 500)
Y_mint.mint_to(Y_b, God, 500)

# alice deposit
deposit_metas = [
AccountMeta(Alice.public_key, True, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 1
amount_a = 10
deposit_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")
instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
tix.add(instruction)
y = client.send_transaction(tix, Alice)

In [58]:
# bob deposit
tix = Transaction()
deposit_metas = [
AccountMeta(Bob.public_key, True, False),
AccountMeta(Y_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(Y_b, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 1
amount_b = 1
deposit_instruction_data = idx.to_bytes(1,"little") + amount_b.to_bytes(8,"little")
instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
tix.add(instruction)
y = client.send_transaction(tix, Bob)

In [59]:
# alice withdraw
tix = Transaction()
withdraw_metas = [
AccountMeta(Alice.public_key, False, False),
AccountMeta(Y_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(Y_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 2
amount_b = 1
withdraw_instruction_data = idx.to_bytes(1,"little") + amount_b.to_bytes(8,"little")
instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
tix.add(instruction)
y = client.send_transaction(tix, Alice)

In [53]:
# bob withdraw
tix = Transaction()
withdraw_metas = [
AccountMeta(Bob.public_key, False, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_b, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 2
amount_a = 10
withdraw_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")
instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
tix.add(instruction)

# send to client
y = client.send_transaction(tix, Bob)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 1: custom program error: 0x1', 'data': {'accounts': None, 'err': {'InstructionError': [1, {'Custom': 1}]}, 'logs': ['Program Fni4pJzEwY3sCE8MAVQFHbXdqm1vEMm85o1o8ormWUJW invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: Made try from slice', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3246 of 189301 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program Fni4pJzEwY3sCE8MAVQFHbXdqm1vEMm85o1o8ormWUJW consumed 15374 of 200000 compute units', 'Program Fni4pJzEwY3sCE8MAVQFHbXdqm1vEMm85o1o8ormWUJW success', 'Program Fni4pJzEwY3sCE8MAVQFHbXdqm1vEMm85o1o8ormWUJW invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Progra